<a href="https://colab.research.google.com/github/bobby-mclaughlinjr/covid/blob/master/Rt*.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime as d

In [0]:
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
COLUMNS = {'Province/State': 'area', 'Country/Region': 'region', 'Lat': 'latitude', 'Long': 'longitude'}

In [0]:
# Default parameters
ASYMPTOMATIC = 0.75
R0 = 2.2
INCUBATION_DURATION = 5.2
INFECTION_DURATION = 2.9

In [0]:
class Rt(object):

    def __init__(self
                 , R0=R0
                 , incubation_duration=INCUBATION_DURATION
                 , infection_duration=INFECTION_DURATION
                 , asymptomatic=ASYMPTOMATIC
                 , data=None
                 ):

        self.R0 = R0
        self.incubation_duration = incubation_duration
        self.infection_duration = infection_duration
        self.asymptomatic = asymptomatic

        self.data = data
        self.transformed_data = None

    @staticmethod
    def smoothing(X, window=7):
        return X.rolling(window=window).mean()

    def get_data(self, source='CSSE'):
        df = pd.read_csv(URL).rename(columns=COLUMNS)
        data = pd.melt(df, id_vars=COLUMNS.values(), var_name='date', value_name='cases')
        data['date'] = [d.datetime.strptime(str(date), '%m/%d/%y') for date in data['date']]

        self.data = data.sort_index().groupby(['region', 'date'])['cases'].sum().loc[['US', 'Spain', 'Italy'], :]

        return self

    def assumed(self, infectious, Rt, shift=7):
        return (Rt / self.infection_duration) * infectious.shift(shift) * (1 - self.asymptomatic)

    def re_calculate(self, smoothing=7):
        return self.calculate().get_historical()

    def calculate(self, smoothing=7):
        self.transformed_data = self.data.copy(deep=True).groupby(level=0).apply(self._calculate, smoothing=smoothing)
        return self

    def _calculate(self, data, smoothing=7):
        data = data.to_frame('cases').reset_index(level=0, drop=True)

        data['total_cases'] = data['cases'] / (1 - self.asymptomatic)
        data['asymptomatic_cases'] = data['total_cases'] - data['cases']

        data['new_cases'] = data['cases'].diff()
        data['new_total_cases'] = data['total_cases'].diff()
        data['new_total_cases_shift'] = data['new_total_cases'].shift(13)

        # Rt
        data['infectious'] = (data['new_total_cases'] - data['new_total_cases'].shift(3)).expanding().apply(lambda x: np.nansum(x))

        data['Rt^'] = self.smoothing(data['new_total_cases'].rolling(window=3).mean() * self.infection_duration / data['infectious'].rolling(window=3).mean().shift(8), window=smoothing)
        data['Rt*'] = self.smoothing(-(data['total_cases'] - data['new_total_cases_shift']) / ((data['total_cases'] - data['new_total_cases_shift']).shift(1)) * np.log(1 / self.incubation_duration) - 1, window=smoothing)
        data['Rt'] = data[['Rt^', 'Rt*']].mean(axis=1)

        data['ex_ante'] = ((data['Rt'] / self.infection_duration) * data['infectious'].shift(6) * (1 - self.asymptomatic)).shift()
        data['ex_post'] = data['new_cases'].shift(-1)

        return data

    def get_historical(self, date=None):
        results = self.transformed_data.copy(deep=True).groupby(level=0).apply(self._get_historical, date=date)
        return pd.DataFrame(results.tolist(), index=results.index)

    @staticmethod
    def _get_historical(data, date=None):
        data.reset_index(level=0, drop=True, inplace=True)

        if date is None:
            current = data.iloc[-1]
            last = data.iloc[-2]
        else:
            loc = data.index.get_loc(date)
            current = data.iloc[loc]
            last = data.iloc[loc - 1]

        return {'current_Rt': current['Rt']
                , 'last_Rt': last['Rt']
                , 'delta_Rt': current['Rt'] - last['Rt']
                , 'projected': current['ex_ante']
                , 'last_projected': last['ex_ante']
                , 'ex_post': last['ex_post']
                , 'error': last['ex_post'] - last['ex_ante']
                , 'error_pct': (last['ex_ante'] - last['ex_post']) / last['ex_post']
                }

In [6]:
Rt().get_data().calculate().get_historical()

,current_Rt,last_Rt,delta_Rt,projected,last_projected,ex_post,error,error_pct
region,,,,,,,,
Italy,0.726579,0.734940,-0.008361,2589.271147,3071.921472,2729.0,-342.921472,0.125658
Spain,0.754320,0.723389,0.030932,2373.213768,2933.220497,3968.0,1034.779503,-0.260781
US,0.821440,0.812480,0.008961,22769.880959,23648.909997,39460.0,15811.090003,-0.400687
